In [0]:
import random
from os import listdir
from PIL import Image
import os
from datetime import datetime
import tensorflow as tf

In [0]:
from google.colab import drive
drive.mount('/content/drive')

ImportError: No module named colab

In [0]:
%cd drive/My Drive/Colab Notebooks/cropped_months/dataset/

## Paths to Training, Testing & Validation

In [0]:
trainInputImagesPath = 'dataset/Training/TrainingInput'
trainOutputImagesPath = 'dataset/Training/TrainingOutput'
testInputImagesPath = 'dataset/Test/TestInput'
testOutputImagesPath = 'dataset/Test/TestOutput'
validInputImagesPath = 'dataset/Validation/ValidationInput'
validOutputImagesPath = 'dataset/Validation/ValidationOutput'

In [0]:
trainInputImagesFiles = listdir(trainInputImagesPath)
trainInputImagesFiles.sort()
trainOutputImagesFiles = listdir(trainOutputImagesPath)
trainOutputImagesFiles.sort()
testInputImagesFiles = listdir(testInputImagesPath)
testInputImagesFiles.sort()
testOutputImagesFiles = listdir(testOutputImagesPath)
testOutputImagesFiles.sort()
validInputImagesFiles = listdir(validInputImagesPath)
validInputImagesFiles.sort()
validOutputImagesFiles = listdir(validOutputImagesPath)
validOutputImagesFiles.sort()

### Number of input vs lables Check

In [0]:
print(str(datetime.now()) + ': trainInputImagesFiles:', len(trainInputImagesFiles))
print(str(datetime.now()) + ': trainOutputImagesFiles:',  len(trainOutputImagesFiles))
if(len(trainInputImagesFiles) != len(trainOutputImagesFiles)):
    raise Exception('train input images and output images number mismatch')

print(str(datetime.now()) + ': testInputImagesFiles:', len(testInputImagesFiles))
print(str(datetime.now()) + ': testOutputImagesFiles:', len(testOutputImagesFiles))
if(len(testInputImagesFiles) != len(testOutputImagesFiles)):
    raise Exception('test input images and output images number mismatch')

print(str(datetime.now()) + ': validInputImagesFiles:', len(validInputImagesFiles))
print(str(datetime.now()) + ': validOutputImagesFiles:', len(validOutputImagesFiles))
if(len(validInputImagesFiles) != len(validOutputImagesFiles)):
    raise Exception('valid input images and output images number mismatch')

('2019-11-12 17:49:35.000336: trainInputImagesFiles:', 1108)
('2019-11-12 17:49:35.000715: trainOutputImagesFiles:', 1108)
('2019-11-12 17:49:35.000967: testInputImagesFiles:', 49)
('2019-11-12 17:49:35.001111: testOutputImagesFiles:', 49)
('2019-11-12 17:49:35.001327: validInputImagesFiles:', 14)
('2019-11-12 17:49:35.001508: validOutputImagesFiles:', 14)


## IF Mismatch, Index Check

In [0]:
for i in range(len(trainInputImagesFiles)):
    inputImageFile = trainInputImagesFiles[i][:-5]
    outputImageFile = trainOutputImagesFiles[i][:-4]
    if(inputImageFile != outputImageFile):
        raise Exception('train inputImageFile and outputImageFile mismatch at index', str(i))

In [0]:
for i in range(len(testInputImagesFiles)):
    inputImageFile = testInputImagesFiles[i][:-5]
    outputImageFile = testOutputImagesFiles[i][:-4]
    if(inputImageFile != outputImageFile):
        raise Exception('test inputImageFile and outputImageFile mismatch at index', str(i))

In [0]:
for i in range(len(validInputImagesFiles)):
    inputImageFile = validInputImagesFiles[i][:-5]
    outputImageFile = validOutputImagesFiles[i][:-4]
    if(inputImageFile != outputImageFile):
        raise Exception('valid inputImageFile and outputImageFile mismatch at index', str(i))


In [0]:
print(str(datetime.now()) + ': input and output files check success')

2019-11-12 17:49:45.425690: input and output files check success


## Writing to Files Subroutine

In [0]:
def writeDataFile(inputImagePath, outputImagePath, inputImageFiles, outputImageFiles, dataFileName):
    dataFile = open(dataFileName, 'w')
    rectSize = 5
    linesCount = 0
    linesLimit = 200000
    linesCountPerImage = 0
    linesLimitPerImage = (linesLimit / len(inputImageFiles)) + 1
    import pdb
    for i in range(len(inputImageFiles)):
        print(str(datetime.now()) + ': prcessing image', i)
        linesCountPerImage = 0
        inputImage = Image.open(inputImagePath + '/' + inputImageFiles[i])
        inputImageXSize, inputImageYSize = inputImage.size
        # inputImagePixels = inputImage.load()
        
        outputImage = Image.open(outputImagePath + '/' + outputImageFiles[i])
        outputImageXSize, outputImageYSize = outputImage.size
        outputImagePixels = outputImage.load()
        
        if((inputImageXSize != outputImageXSize) or (inputImageYSize != outputImageYSize)):
            raise Exception('train inputImage and outputImage mismatch at index', str(i))

        outputImageRoadPixelsArr = [];
        outputImageNonRoadPixelsArr= [];
        import pdb
        for x in range(rectSize//2, inputImageXSize - (rectSize//2)):
            for y in range(rectSize//2, inputImageYSize - (rectSize//2)):
                
                isRoadPixel = outputImagePixels[x, y]
                if(isRoadPixel):
                    
                    outputImageRoadPixelsArr.append((x, y))
                else:
                    outputImageNonRoadPixelsArr.append((x, y))
                   

        random.shuffle(outputImageRoadPixelsArr)
        random.shuffle(outputImageNonRoadPixelsArr)
        
        for m in range(len(outputImageRoadPixelsArr)):
            if(linesCountPerImage >= linesLimitPerImage):
                break
            
            if(((m*2) + 1) >= len(outputImageNonRoadPixelsArr)):
                break
            
            x = outputImageRoadPixelsArr[m][0];
            y = outputImageRoadPixelsArr[m][1];
            
            rect = (x - (rectSize//2), y - (rectSize//2), x + (rectSize//2) + 1, y + (rectSize//2) + 1)
            subImage = inputImage.crop(rect).load()
            line = ''
            for i in range(rectSize):
                for j in range(rectSize):
                    line += str(subImage[i, j][0]) + ','
                    line += str(subImage[i, j][1]) + ','
                    line += str(subImage[i, j][2]) + ','
            
            line += str(1) + '\n'
            linesCount += 1
            linesCountPerImage += 1
            dataFile.write(line)
            
            for n in range(2):
                x = outputImageNonRoadPixelsArr[(m*2) + n][0];
                y = outputImageNonRoadPixelsArr[(m*2) + n][1];
                
                rect = (x - (rectSize//2), y - (rectSize//2), x + (rectSize//2) + 1, y + (rectSize//2) + 1)
                subImage = inputImage.crop(rect).load()
                line = ''
                for i in range(rectSize):
                    for j in range(rectSize):
                        line += str(subImage[i, j][0]) + ','
                        line += str(subImage[i, j][1]) + ','
                        line += str(subImage[i, j][2]) + ','
                
                line += str(0) + '\n'
                linesCount += 1
                linesCountPerImage += 1
                dataFile.write(line)
    
    print(str(datetime.now()) + ': ' + dataFileName + ' linesCount:', linesCount)

## Creating Files

In [0]:
trainDataFileName = 'dataset/train.csv'
testDataFileName = 'dataset/test.csv'
validDataFileName = 'dataset/valid.csv'

In [0]:
print(str(datetime.now()) + ': writing trainDataFile')
writeDataFile(trainInputImagesPath, trainOutputImagesPath, trainInputImagesFiles, trainOutputImagesFiles, trainDataFileName)
print(str(datetime.now()) + ': trainDataFile complete')

2019-11-12 17:49:57.839961: writing trainDataFile
('2019-11-12 17:49:57.840184: prcessing image', 0)
('2019-11-12 17:49:59.594548: prcessing image', 1)
('2019-11-12 17:50:02.548873: prcessing image', 2)
('2019-11-12 17:50:05.266410: prcessing image', 3)
('2019-11-12 17:50:09.932993: prcessing image', 4)
('2019-11-12 17:50:14.025930: prcessing image', 5)
('2019-11-12 17:50:18.556835: prcessing image', 6)
('2019-11-12 17:50:23.265118: prcessing image', 7)
('2019-11-12 17:50:27.284195: prcessing image', 8)
('2019-11-12 17:50:31.081234: prcessing image', 9)
('2019-11-12 17:50:35.683239: prcessing image', 10)
('2019-11-12 17:50:39.119139: prcessing image', 11)
('2019-11-12 17:50:42.786358: prcessing image', 12)
('2019-11-12 17:50:45.986091: prcessing image', 13)
('2019-11-12 17:50:49.758924: prcessing image', 14)
('2019-11-12 17:50:55.229182: prcessing image', 15)
('2019-11-12 17:50:58.471519: prcessing image', 16)
('2019-11-12 17:51:02.381982: prcessing image', 17)
('2019-11-12 17:51:05.52

('2019-11-12 17:58:09.373225: prcessing image', 156)
('2019-11-12 17:58:12.584843: prcessing image', 157)
('2019-11-12 17:58:15.552767: prcessing image', 158)
('2019-11-12 17:58:18.828949: prcessing image', 159)
('2019-11-12 17:58:23.358155: prcessing image', 160)
('2019-11-12 17:58:26.633768: prcessing image', 161)
('2019-11-12 17:58:29.966353: prcessing image', 162)
('2019-11-12 17:58:33.227886: prcessing image', 163)
('2019-11-12 17:58:37.202115: prcessing image', 164)
('2019-11-12 17:58:41.125046: prcessing image', 165)
('2019-11-12 17:58:44.067921: prcessing image', 166)
('2019-11-12 17:58:46.622318: prcessing image', 167)
('2019-11-12 17:58:49.190066: prcessing image', 168)
('2019-11-12 17:58:51.739582: prcessing image', 169)
('2019-11-12 17:58:54.399354: prcessing image', 170)
('2019-11-12 17:58:57.004342: prcessing image', 171)
('2019-11-12 17:58:59.679689: prcessing image', 172)
('2019-11-12 17:59:02.272300: prcessing image', 173)
('2019-11-12 17:59:04.949929: prcessing image'

('2019-11-12 18:06:07.026644: prcessing image', 311)
('2019-11-12 18:06:09.668906: prcessing image', 312)
('2019-11-12 18:06:13.176238: prcessing image', 313)
('2019-11-12 18:06:17.716095: prcessing image', 314)
('2019-11-12 18:06:20.181600: prcessing image', 315)
('2019-11-12 18:06:24.491827: prcessing image', 316)
('2019-11-12 18:06:27.146229: prcessing image', 317)
('2019-11-12 18:06:31.541094: prcessing image', 318)
('2019-11-12 18:06:34.402212: prcessing image', 319)
('2019-11-12 18:06:37.124627: prcessing image', 320)
('2019-11-12 18:06:41.588387: prcessing image', 321)
('2019-11-12 18:06:44.153670: prcessing image', 322)
('2019-11-12 18:06:46.701678: prcessing image', 323)
('2019-11-12 18:06:49.063427: prcessing image', 324)
('2019-11-12 18:06:51.609634: prcessing image', 325)
('2019-11-12 18:06:54.456054: prcessing image', 326)
('2019-11-12 18:06:59.345158: prcessing image', 327)
('2019-11-12 18:07:03.850342: prcessing image', 328)
('2019-11-12 18:07:07.148776: prcessing image'

('2019-11-12 18:14:23.725460: prcessing image', 466)
('2019-11-12 18:14:26.951888: prcessing image', 467)
('2019-11-12 18:14:30.027730: prcessing image', 468)
('2019-11-12 18:14:34.095753: prcessing image', 469)
('2019-11-12 18:14:37.003010: prcessing image', 470)
('2019-11-12 18:14:40.170011: prcessing image', 471)
('2019-11-12 18:14:43.722303: prcessing image', 472)
('2019-11-12 18:14:47.380387: prcessing image', 473)
('2019-11-12 18:14:51.970153: prcessing image', 474)
('2019-11-12 18:14:56.214500: prcessing image', 475)
('2019-11-12 18:14:59.597523: prcessing image', 476)
('2019-11-12 18:15:02.994966: prcessing image', 477)
('2019-11-12 18:15:05.916334: prcessing image', 478)
('2019-11-12 18:15:09.827787: prcessing image', 479)
('2019-11-12 18:15:13.795086: prcessing image', 480)
('2019-11-12 18:15:17.669166: prcessing image', 481)
('2019-11-12 18:15:21.212392: prcessing image', 482)
('2019-11-12 18:15:24.652572: prcessing image', 483)
('2019-11-12 18:15:28.699814: prcessing image'

('2019-11-12 18:22:09.633779: prcessing image', 621)
('2019-11-12 18:22:14.340482: prcessing image', 622)
('2019-11-12 18:22:19.430648: prcessing image', 623)
('2019-11-12 18:22:24.608150: prcessing image', 624)
('2019-11-12 18:22:30.374666: prcessing image', 625)
('2019-11-12 18:22:35.309532: prcessing image', 626)
('2019-11-12 18:22:40.186105: prcessing image', 627)
('2019-11-12 18:22:43.157659: prcessing image', 628)
('2019-11-12 18:22:48.352771: prcessing image', 629)
('2019-11-12 18:22:52.944816: prcessing image', 630)
('2019-11-12 18:22:57.874617: prcessing image', 631)
('2019-11-12 18:23:02.636581: prcessing image', 632)
('2019-11-12 18:23:07.952415: prcessing image', 633)
('2019-11-12 18:23:11.254233: prcessing image', 634)
('2019-11-12 18:23:16.323193: prcessing image', 635)
('2019-11-12 18:23:21.163559: prcessing image', 636)
('2019-11-12 18:23:25.864756: prcessing image', 637)
('2019-11-12 18:23:30.616168: prcessing image', 638)
('2019-11-12 18:23:35.437307: prcessing image'

('2019-11-12 18:34:18.753251: prcessing image', 776)
('2019-11-12 18:34:24.014458: prcessing image', 777)
('2019-11-12 18:34:28.658523: prcessing image', 778)
('2019-11-12 18:34:33.405826: prcessing image', 779)
('2019-11-12 18:34:36.326754: prcessing image', 780)
('2019-11-12 18:34:40.950261: prcessing image', 781)
('2019-11-12 18:34:45.551264: prcessing image', 782)
('2019-11-12 18:34:50.357779: prcessing image', 783)
('2019-11-12 18:34:55.227958: prcessing image', 784)
('2019-11-12 18:35:00.061433: prcessing image', 785)
('2019-11-12 18:35:04.952480: prcessing image', 786)
('2019-11-12 18:35:09.597110: prcessing image', 787)
('2019-11-12 18:35:14.214960: prcessing image', 788)
('2019-11-12 18:35:18.975578: prcessing image', 789)
('2019-11-12 18:35:23.745779: prcessing image', 790)
('2019-11-12 18:35:26.537846: prcessing image', 791)
('2019-11-12 18:35:31.544133: prcessing image', 792)
('2019-11-12 18:35:36.468393: prcessing image', 793)
('2019-11-12 18:35:41.005336: prcessing image'

('2019-11-12 18:43:34.173041: prcessing image', 931)
('2019-11-12 18:43:37.538971: prcessing image', 932)
('2019-11-12 18:43:40.289995: prcessing image', 933)
('2019-11-12 18:43:43.158866: prcessing image', 934)
('2019-11-12 18:43:46.242301: prcessing image', 935)
('2019-11-12 18:43:49.444221: prcessing image', 936)
('2019-11-12 18:43:52.255021: prcessing image', 937)
('2019-11-12 18:43:54.971211: prcessing image', 938)
('2019-11-12 18:43:57.622747: prcessing image', 939)
('2019-11-12 18:44:00.210102: prcessing image', 940)
('2019-11-12 18:44:02.976455: prcessing image', 941)
('2019-11-12 18:44:07.236247: prcessing image', 942)
('2019-11-12 18:44:09.785540: prcessing image', 943)
('2019-11-12 18:44:13.334824: prcessing image', 944)
('2019-11-12 18:44:15.910944: prcessing image', 945)
('2019-11-12 18:44:20.297338: prcessing image', 946)
('2019-11-12 18:44:22.860520: prcessing image', 947)
('2019-11-12 18:44:27.523127: prcessing image', 948)
('2019-11-12 18:44:31.890754: prcessing image'

('2019-11-12 18:52:50.427309: prcessing image', 1084)
('2019-11-12 18:52:54.535590: prcessing image', 1085)
('2019-11-12 18:52:58.797805: prcessing image', 1086)
('2019-11-12 18:53:02.209098: prcessing image', 1087)
('2019-11-12 18:53:05.615696: prcessing image', 1088)
('2019-11-12 18:53:09.703055: prcessing image', 1089)
('2019-11-12 18:53:13.058391: prcessing image', 1090)
('2019-11-12 18:53:16.401936: prcessing image', 1091)
('2019-11-12 18:53:19.377682: prcessing image', 1092)
('2019-11-12 18:53:22.697037: prcessing image', 1093)
('2019-11-12 18:53:25.215483: prcessing image', 1094)
('2019-11-12 18:53:27.959575: prcessing image', 1095)
('2019-11-12 18:53:31.533304: prcessing image', 1096)
('2019-11-12 18:53:34.318224: prcessing image', 1097)
('2019-11-12 18:53:37.432790: prcessing image', 1098)
('2019-11-12 18:53:40.458079: prcessing image', 1099)
('2019-11-12 18:53:44.017400: prcessing image', 1100)
('2019-11-12 18:53:47.681011: prcessing image', 1101)
('2019-11-12 18:53:50.841392

In [0]:

print(str(datetime.now()) + ': writing testDataFile')
writeDataFile(testInputImagesPath, testOutputImagesPath, testInputImagesFiles, testOutputImagesFiles, testDataFileName)
print(str(datetime.now()) + ': testDataFile complete')


2019-11-12 18:54:14.387268: writing testDataFile
('2019-11-12 18:54:15.280966: prcessing image', 0)
('2019-11-12 18:54:17.754809: prcessing image', 1)
('2019-11-12 18:54:21.579185: prcessing image', 2)
('2019-11-12 18:54:24.870429: prcessing image', 3)
('2019-11-12 18:54:28.396521: prcessing image', 4)
('2019-11-12 18:54:32.384183: prcessing image', 5)
('2019-11-12 18:54:35.737299: prcessing image', 6)
('2019-11-12 18:54:39.646428: prcessing image', 7)
('2019-11-12 18:54:43.177518: prcessing image', 8)
('2019-11-12 18:54:48.356102: prcessing image', 9)
('2019-11-12 18:54:53.506654: prcessing image', 10)
('2019-11-12 18:54:57.850882: prcessing image', 11)
('2019-11-12 18:55:02.846581: prcessing image', 12)
('2019-11-12 18:55:07.503877: prcessing image', 13)
('2019-11-12 18:55:16.988814: prcessing image', 14)
('2019-11-12 18:55:23.970933: prcessing image', 15)
('2019-11-12 18:55:27.299864: prcessing image', 16)
('2019-11-12 18:55:34.826501: prcessing image', 17)
('2019-11-12 18:55:39.704

In [0]:

print(str(datetime.now()) + ': writing validDataFile')
writeDataFile(validInputImagesPath, validOutputImagesPath, validInputImagesFiles, validOutputImagesFiles, validDataFileName)
print(str(datetime.now()) + ': validDataFile complete')

2019-11-12 18:58:13.291767: writing validDataFile
('2019-11-12 18:58:13.292138: prcessing image', 0)
('2019-11-12 18:58:15.663531: prcessing image', 1)
('2019-11-12 18:58:18.361512: prcessing image', 2)
('2019-11-12 18:58:21.123425: prcessing image', 3)
('2019-11-12 18:58:23.731308: prcessing image', 4)
('2019-11-12 18:58:26.421242: prcessing image', 5)
('2019-11-12 18:58:29.186733: prcessing image', 6)
('2019-11-12 18:58:31.851049: prcessing image', 7)
('2019-11-12 18:58:34.455001: prcessing image', 8)
('2019-11-12 18:58:37.167576: prcessing image', 9)
('2019-11-12 18:58:40.067349: prcessing image', 10)
('2019-11-12 18:58:42.829231: prcessing image', 11)
('2019-11-12 18:58:45.495232: prcessing image', 12)
('2019-11-12 18:58:48.174360: prcessing image', 13)
('2019-11-12 18:58:51.152011: dataset/valid.csv linesCount:', 200004)
2019-11-12 18:58:51.478411: validDataFile complete


# Training and Testing

In [0]:
from datetime import datetime
import numpy as np
import tensorflow as tf

print(str(datetime.now()) + ': loading data files')
# Data sets
trainDataFileName = 'dataset/train.csv'
testDataFileName = 'dataset/test.csv'
validationDataFileName = 'dataset/valid.csv'
# Load datasets.
trainData = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=trainDataFileName,
    target_dtype=np.int,
    features_dtype=np.int)
testData = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=testDataFileName,
    target_dtype=np.int,
    features_dtype=np.int)
# validationData = tf.contrib.learn.datasets.base.load_csv_without_header(
#     filename=validationDataFileName,
#     target_dtype=np.int,
#     features_dtype=np.int)

trainingSteps = 1000
totalTrainingSteps = 5000

featureColumns = [tf.contrib.layers.real_valued_column("", dimension=75)]
hiddenUnits = [100, 150, 100, 50]
classes = 2
modelDir = 'model'
classifierConfig = tf.contrib.learn.RunConfig(save_checkpoints_secs = None,
                                              save_checkpoints_steps = trainingSteps)

classifier = tf.contrib.learn.DNNClassifier(feature_columns = featureColumns,
                                                hidden_units = hiddenUnits,
                                                n_classes = classes,
                                                model_dir = modelDir,
                                                config = classifierConfig)

# Define the training inputs
def getTrainData():
    x = tf.constant(trainData.data)
    y = tf.constant(trainData.target)
    return x, y

# Define the test inputs
def getTestData():
    x = tf.constant(testData.data)
    y = tf.constant(testData.target)
    return x, y

# Define the validation inputs
# def getValidationData():
#     x = tf.constant(validationData.data)
#     y = tf.constant(validationData.target)
#     return x, y

# print(str(datetime.now()) + ': training...')
# prevAccuracy = -1.0
# for i in range(totalTrainingSteps // trainingSteps):
#     classifier.fit(input_fn=getTrainData, steps=trainingSteps)
#     currentAccuracy = classifier.evaluate(input_fn=getValidationData, steps=1)['accuracy']
#     print(str(datetime.now()) + ': training: validation step: ' + str(i) + ' currentAccuracy:', currentAccuracy)
#     #if(currentAccuracy <= prevAccuracy):
#     #    break
#     prevAccuracy = currentAccuracy

print(str(datetime.now()) + ': training...')
classifier.fit(input_fn=getTrainData, steps=totalTrainingSteps)
print(str(datetime.now()) + ': testing...')
accuracy = classifier.evaluate(input_fn=getTestData, steps=1)['accuracy']
print(str(datetime.now()) + ': done')
print(str(datetime.now()) + ': accuracy:', accuracy)

2019-11-12 20:02:40.397786: loading data files
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd62c66d610>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_protocol': None, '_save_checkpoints_steps': 1000, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_model_dir': 'model', '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_environment': 'local', '_save_summary_steps': 100}
2019-11-12 20:02:54.181504: training...
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:t

## Accuracy = 82 %

# Test with seperate Input to Generate Output Mask

In [0]:
from datetime import datetime
from PIL import Image
import sys
import numpy as np
import tensorflow as tf

if(len(sys.argv) != 3):
	raise Exception('invalid command line arguments')

print(str(datetime.now()) + ': initializing input data...')

rectSize = 5;

inputImagePath = '/home/obiii/Desktop/Thesis/Roads-Segmentation-Mnih-Dataset-/TestInput'
inputImageFile = '23429080_15.tiff'
inputImage = Image.open(inputImagePath + '/' + inputImageFile)
inputImageXSize, inputImageYSize = inputImage.size

outputImagePath = '/home/obiii/Desktop/Thesis/Roads-Segmentation-Mnih-Dataset-/TestOutput'
outputImageFile = 'Output'
outputImage = inputImage.crop((rectSize//2, rectSize//2, inputImageXSize - (rectSize//2), inputImageYSize - (rectSize//2)))
outputImageXSize, outputImageYSize = outputImage.size

print(str(datetime.now()) + ': initializing model...')
featureColumns = [tf.contrib.layers.real_valued_column("", dimension=75)]
# hiddenUnits = [100, 100, 100, 50]
# hiddenUnits = [100, 150, 200, 150, 100, 50]
hiddenUnits = [100, 150, 100, 50]
classes = 2
classifier = tf.contrib.learn.DNNClassifier(feature_columns = featureColumns,
												hidden_units = hiddenUnits,
												n_classes = classes,
												model_dir = 'model')

def extractFeatures():
    features = np.zeros((((inputImageXSize - ((rectSize//2)*2)) * (inputImageYSize - ((rectSize//2)*2))), rectSize*rectSize*3), dtype=np.int)
    rowIndex = 0
    
    for x in range(rectSize//2, inputImageXSize - (rectSize//2)):
        for y in range(rectSize//2, inputImageYSize - (rectSize//2)):            
            rect = (x - (rectSize//2), y - (rectSize//2), x + (rectSize//2) + 1, y + (rectSize//2) + 1)
            subImage = inputImage.crop(rect).load()
            colIndex = 0
            for i in range(rectSize):
                for j in range(rectSize):
                    features[rowIndex, colIndex] = subImage[i, j][0]
                    colIndex += 1
                    features[rowIndex, colIndex] = subImage[i, j][1]
                    colIndex += 1
                    features[rowIndex, colIndex] = subImage[i, j][2]
                    colIndex += 1
            
            rowIndex += 1
    
    return features
    
def constructOutputImage(predictions):
    outputImagePixels = outputImage.load()
    rowIndex = 0
    for x in range(outputImageXSize):
        for y in range(outputImageYSize):
            outputImagePixels[x, y] = ((255, 255, 255) if predictions[rowIndex] else (0, 0, 0))
            rowIndex += 1
        
print(str(datetime.now()) + ': processing image', inputImageFile)
predictions = list(classifier.predict(input_fn=extractFeatures))

print(str(datetime.now()) + ': constructing output image...')
constructOutputImage(predictions)

print(str(datetime.now()) + ': saving output image...')
outputImage.save(outputImagePath + '/' + outputImageFile, 'JPEG')

print(str(datetime.now()) + ': done')